In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import spacy
import re

# Read recipe inputs
latest_transcription = dataiku.Dataset("transcription_latest")
latest_transcription_df = latest_transcription.get_dataframe()

conversations_diarisees = latest_transcription_df['text_diarise']
liste_de_conversations_diarisees = list(conversations_diarisees)

# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

#synthese_transcription_df = latest_transcription_df # For this sample code, simply copy input to output


# Write recipe outputs
#synthese_transcription = dataiku.Dataset("synthese_transcription")
#synthese_transcription.write_with_schema(synthese_transcription_df)

In [ ]:
def nettoie_resume(texte):

    l = []
    for i in texte:
        if "s'il vous plaît" in i:
            pass
        else:
            l.append(i)

    l2 = []
    for i in l:
        if "bonjour que puis-je faire pour vous" in i:
            pass
        else:
            l2.append(i)



    l3 = []
    for i in l2:
        if "votre nom" in i:
            pass
        else:
            l3.append(i)

    l4 = []
    for i in l3:
        if "au revoir" in i:
            pass
        else:
            l4.append(i)

    l5 = []
    for i in l4:
        if "bonne journée" in i:
            pass
        else:
            l5.append(i)

    l6 = []
    for i in l5:
        if "adresse postale" in i:
            pass
        else:
            l6.append(i)

    return  l6

In [ ]:
liste_des_tours_de_paroles_par = []
for convers in liste_de_conversations_diarisees:
    str_conversation=str(convers)
    str_conversation = str_conversation.replace('[pers.pre]', '[pers pre]')   #supprimer des NER
    str_conversation = str_conversation.replace('[pers.nom]', '[pers nom]')
    str_conversation = str_conversation.replace('[pers.nom.epl]', '[pers nom epl]')
    str_conversation = str_conversation.replace('[num tel]', '[num tel]')
    str_conversation = str_conversation.replace('[loc.voie]', '[loc voie]')
    str_conversation = str_conversation.replace('[loc.cp]', '[loc cp]')
    str_conversation_vf = str_conversation.replace('[pers.mail]', '[pers mail]')
    conversation_intermediaire = str_conversation_vf.split(".")
    liste_des_tours_de_paroles_par.append(conversation_intermediaire)
#print(len(liste_des_tours_de_paroles_par[0]))
#print(liste_des_tours_de_paroles_par)
print(len(liste_des_tours_de_paroles_par), 'conservations')#récupère la liste des conversations entières

liste_des_tours = []

for conver in liste_des_tours_de_paroles_par:
    a = nettoie_resume(conver)
    liste_des_tours.append(a)

In [ ]:
id_conv = latest_transcription_df['nom']
#créer un dataframe pour output
new_df = pd.DataFrame(columns = ['nom','synthese'])
new_df['nom'] = id_conv

In [ ]:
def resume_all (liste):
    nlp = spacy.load('fr_core_news_md')

    matrice_des_conversations_tokenizes = []
    matrice_des_conversations_vectorizes = []
    compteur = 0
    for conversation in liste :
        compteur+=1
        if len(liste)%compteur == 0:
            print("en cours...")
        array_des_tours_de_paroles_vectorizes = []
        array_des_tours_de_paroles_tokenizes = []
        for tour_de_parole in conversation:
            tour_de_parole_vectorize = []
            tour_de_parole_tokenize =[]
            tour_de_parole_spacy = nlp(tour_de_parole)
            for token in tour_de_parole_spacy:
                tour_de_parole_vectorize.append(token.vector_norm if token.has_vector == True else 0)#padding
                tour_de_parole_tokenize.append(token)
            array_des_tours_de_paroles_vectorizes.append(tour_de_parole_vectorize)
            array_des_tours_de_paroles_tokenizes.append(tour_de_parole_tokenize)
        matrice_des_conversations_vectorizes.append(array_des_tours_de_paroles_vectorizes)
        matrice_des_conversations_tokenizes.append(array_des_tours_de_paroles_tokenizes)
    len(matrice_des_conversations_vectorizes) == len(matrice_des_conversations_tokenizes)
    from sklearn.cluster import KMeans
    from sklearn.metrics import pairwise_distances_argmin_min
    list_summary = []
    for i in range(len(np.array(matrice_des_conversations_vectorizes))):
        doc_vec = []
        document = np.array(matrice_des_conversations_vectorizes[i])


        for tdp in document:

            tdp_vec = np.zeros((100))
            for vk in tdp:

                tdp_vec = np.append(tdp_vec,vk)  #ajouter les valeurs de document
                tdp_vec = np.delete(tdp_vec, 0)  #supprimer 0

            doc_vec.append(tdp_vec)

        if len(doc_vec) >= 25:

            n_clusters = 25

            kmeans = KMeans(n_clusters=n_clusters)
            means = kmeans.fit(doc_vec)
            avg = []




            docu = liste[i]
            #print(doc)



            for j in range(n_clusters):
                idx = np.where(kmeans.labels_ == j)[0]
                avg.append(np.mean(idx))
            #print(avg)


            closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, doc_vec)
            ordering = sorted(range(n_clusters), key=lambda k: avg[k])

            #print(ordering)
            liste_cl=[]
            for p in ordering:
                cl = closest[p]
                liste_cl.append(cl)
            liste_cl = sorted(liste_cl)
            #print(liste_cl)
            #print(document[10])


            summary = '//'.join(docu[idx] for idx in liste_cl)

            list_summary.append(summary)

        else:
            list_summary.append(liste[i])

    return list_summary

In [ ]:
new_df['synthese'] = resume_all(liste_des_tours)
synthese_transcription = dataiku.Dataset("synthese_transcription")
synthese_transcription.write_with_schema(new_df)